In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import * 
from fastai import * 
import pandas as pd 
import numpy as np

In [ ]:
train_data = Path('../input/cassava-leaf-disease-classification/train_images')
test_data = Path('../input/cassava-leaf-disease-classification/test_images')


In [ ]:
# Getting the labels 

df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
df

In [ ]:
# Getting the filenames 

fns_train = get_image_files(train_data)
fns_test = get_image_files(test_data)

len(fns_train) , len(fns_test)

In [ ]:
fns_train

In [ ]:
# Transforms 

item_tfms = RandomResizedCrop(460, min_scale=0.75),
batch_tfms = [*aug_transforms(size=224, max_warp=0. , max_zoom= 0.8 ), Normalize.from_stats(*imagenet_stats)]

In [ ]:
# Building a DataBlock 


cassava = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_x=ColReader(0, pref=train_data),
                   splitter=RandomSplitter(),
                   get_y=ColReader(1),
                   item_tfms = item_tfms,
                   batch_tfms = batch_tfms ) 

In [ ]:
cassava.summary(df)

In [ ]:
# Putting into a dataloader 

train_dls = cassava.dataloaders(df , bs = 32)
train_dls.show_batch(figsize = (15 , 9))

In [ ]:

# Creating a learner 

# learn = cnn_learner(train_dls , alexnet  , metrics = [error_rate , accuracy] , pretrained = False )

In [ ]:
learn_inf = load_learner('../input/vgg-19/export_2.pkl', cpu = False)
learn_inf

In [ ]:
# Creating a test dataloader 

test_dl = train_dls.test_dl(fns_test)

In [ ]:
# Getting predictions with inference 
test_pred = learn_inf.get_preds(dl = test_dl)
test_pred

In [ ]:
# Getting in the Sample Submission File 
sample_sub = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
sample_sub

In [ ]:
# Getting our predictions inside a dataframe

test_pred_max = test_pred[0].argmax(dim=1)
pred_labels = [train_dls.vocab[o] for o in test_pred_max]
sub = pd.DataFrame({'Image_Id':test_dl.items,'label':pred_labels})
sub

In [ ]:
sub['Image_Id'] = sub['Image_Id'].astype(str).str.replace('../input/cassava-leaf-disease-classification/test_images/' , '')
sub

In [ ]:
sub.to_csv('submission.csv' , index=False)